<a href="https://colab.research.google.com/github/MohamedSameh410/DC_GAN/blob/main/DC__GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

Import dataset from kaggle

In [49]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (4).json


{'kaggle.json': b'{"username":"mohamedsameh410","key":"3d112c89fff8060b634a255eca9c03af"}'}

In [50]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [51]:
!kaggle datasets download ashwingupta3012/human-faces

human-faces.zip: Skipping, found more recently modified local copy (use --force to force download)


In [52]:
!unzip human-faces.zip

Archive:  human-faces.zip
replace Humans/1 (1).jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Importing tensornoard

In [53]:
!pip install tensorboard
%load_ext tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Discriminator's implementation

In [54]:
class Discriminator (nn.Module):
  def __init__(self, channels_img, features_d) :
    super(Discriminator, self).__init__()
    self.disc = nn.Sequential(
        #input: Nx64x64x3
        #applying (N+2p-f)/stride+1
        nn.Conv2d(
            channels_img, features_d, kernel_size=4, stride=2, padding=1
        ), #32x32
        nn.LeakyReLU(0.2),
        self._block(features_d, features_d * 2, 4,2,1), #16x16
        self._block(features_d * 2, features_d * 4, 4,2,1), #8x8
        self._block(features_d * 4, features_d * 8, 4,2,1), #4x4
        nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),#1x1
        nn.Sigmoid(),
    )

  def _block(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias=False,
        ),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2),
    )
  def forward(self, x):
    return self.disc(x)

Generator's implementation

In [55]:
class Generator(nn.Module):
  def __init__(self, z_dim, channels_img, features_g) :
    super(Generator, self).__init__()
    self.gen = nn.Sequential(
        #Input: N x z_dim x 1 x 1
        self._block(z_dim, features_g * 16, 4, 1, 0), #4x4
        self._block(features_g * 16, features_g * 8, 4, 2, 1), #8x8
        self._block(features_g * 8, features_g * 4, 4, 2, 1), #16x16
        self._block(features_g * 4, features_g * 2, 4, 2, 1), #32x32
        nn.ConvTranspose2d(
            features_g * 2, channels_img, kernel_size=4, stride=2, padding=1,
        ), #64x64x3
        nn.Tanh(), #[-1, 1]
    )
  def _block(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.ConvTranspose2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias=False,
        ),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
    )
  def forward(self, x):
    x = x + torch.randn_like(x) * 0.1
    return self.gen(x)

Initialize Weights

In [56]:
def initialize_weights(model):
  for m in model.modules():
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data, 0.0, 0.02)

Testing Models

In [57]:
def test():
  N, in_channels, H, W = 8, 3, 64, 64
  z_dim = 100
  x= torch.randn((N, in_channels, H, W))
  disc = Discriminator(in_channels, 8)
  initialize_weights(disc)
  assert disc(x).shape == (N, 1, 1, 1)
  gen = Generator(z_dim, in_channels, 8)
  initialize_weights(gen)
  z = torch.randn((N, z_dim, 1, 1))
  assert gen(z).shape == (N, in_channels, H, W)
  print("models do not have errors")

In [58]:
test()

models do not have errors


Hyperparameters

In [59]:
#sets the device where the tensors and models will be allocated.
#or check if gpu is free will run the model on it if not will run the model on cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Learning_Rate = 2e-4
Batch_Size = 32
Image_size = 64
Channels_img = 3
Z_dim = 100
Num_Epochs = 20
Features_Disc = 64
Features_Gen = 64
#applying transformations together on the input images
transforms = transforms.Compose(
    [
        transforms.Resize((64, 64)),
     transforms.ToTensor(),# A tensor is a multi-dimensional array that can be used as input to a neural network
     transforms.Normalize(
         [0.5 for _ in range(Channels_img)], [0.5 for _ in range(Channels_img)],#mean 0.5 and a standard deviation 0.5
     )
    ]
)

Loading the dataset

In [60]:
dataset = datasets.ImageFolder(root="/content/Dataset",transform=transforms)
data_loader = DataLoader(dataset, batch_size=Batch_Size, shuffle=True)

Training

In [ ]:
gen = Generator(Z_dim, Channels_img,Features_Gen).to(device)
disc = Discriminator(Channels_img, Features_Disc).to(device)
initialize_weights(gen)
initialize_weights(disc)

#optimizers
optim_gen = optim.Adam(gen.parameters(), lr=Learning_Rate, betas=(0.5, 0.999))
optim_disc = optim.Adam(disc.parameters(), lr=Learning_Rate, betas=(0.5, 0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(32, Z_dim, 1, 1).to(device)
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")
step = 0

gen.train()
disc.train()

for epoch in range(Num_Epochs):
  for batch_idx, (real, _) in enumerate(data_loader):
    real = real.to(device)
    noise = torch.randn((Batch_Size, Z_dim, 1, 1)).to(device)
    fake = gen(noise)

    #Train disc
    disc_real = disc(real).reshape(-1)
    loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
    disc_fake = disc(fake).reshape(-1)
    loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
    loss_disc = (loss_disc_real + loss_disc_fake) / 2
    disc.zero_grad()
    loss_disc.backward(retain_graph = True)
    optim_disc.step()

    #Train gen
    output = disc(fake).reshape(-1)
    loss_gen = criterion(output, torch.ones_like(output))
    gen.zero_grad()
    loss_gen.backward()
    optim_gen.step()
  
    #print losses accasionally and print to tensorboard
    if batch_idx % 100 ==0:
      print(f"Epoch [{epoch}/{Num_Epochs}] Batch {batch_idx}/{len(data_loader)} \
            Loss D: {loss_disc:.4f}, Loss G: {loss_gen:.4f}")
      with torch.no_grad():
        fake = gen(fixed_noise)
        # take out (up to) 32 examples
        img_grid_real = torchvision.utils.make_grid(
            real[:32], normalize=True
        )
        img_grid_fake = torchvision.utils.make_grid(
            fake[:32], normalize=True
        )

        writer_real.add_image("Real", img_grid_real, global_step=step)
        writer_fake.add_image("Fake", img_grid_fake, global_step=step)
      step += 1

Epoch [0/20] Batch 0/226             Loss D: 0.6927, Loss G: 0.8084


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [0/20] Batch 100/226             Loss D: 0.0160, Loss G: 4.2088
Epoch [0/20] Batch 200/226             Loss D: 0.2861, Loss G: 2.3531
Epoch [1/20] Batch 0/226             Loss D: 0.4407, Loss G: 2.4641
Epoch [1/20] Batch 100/226             Loss D: 0.5913, Loss G: 2.0812
Epoch [1/20] Batch 200/226             Loss D: 0.6116, Loss G: 2.4120
Epoch [2/20] Batch 0/226             Loss D: 0.5731, Loss G: 1.7682
Epoch [2/20] Batch 100/226             Loss D: 0.8063, Loss G: 2.0064
Epoch [2/20] Batch 200/226             Loss D: 0.5169, Loss G: 1.7382
Epoch [3/20] Batch 0/226             Loss D: 0.5154, Loss G: 1.4813
Epoch [3/20] Batch 100/226             Loss D: 0.5569, Loss G: 2.3393
Epoch [3/20] Batch 200/226             Loss D: 0.5892, Loss G: 2.1470
Epoch [4/20] Batch 0/226             Loss D: 0.5406, Loss G: 1.7643
Epoch [4/20] Batch 100/226             Loss D: 0.5376, Loss G: 2.4717
Epoch [4/20] Batch 200/226             Loss D: 0.4589, Loss G: 1.2705
Epoch [5/20] Batch 0/226    

In [1]:
%tensorboard --logdir=/content/logs

UsageError: Line magic function `%tensorboard` not found.
